# Poker Combinatorics

Since this part of the project was taking a long while we thought of documenting it.

Until we venture into calculating number of cases in different states we should get the basic initial combinatorics down, they may be used as a template for further calculations and it supposedly less complex.

In [1]:
from math import comb

In [3]:
# Royal Flushes
rf_cases = 4 * comb(47, 2)
rf_cases # Correct

4324

In [4]:
# Straight Flushes
sf_cases = 9 * 4 * comb(46, 2)
sf_cases # Correct

37260

In [80]:
# Flushes
fl_cases = 4 * (comb(13, 5) * comb(39, 2) + comb(13, 6) * comb(39, 1) + comb(13, 7))
fl_cases = fl_cases - rf_cases - sf_cases
fl_cases # Correct

4047644

In [34]:
# Straights
st_cases = 0

# Interpretation for the loop: We start from the best straight and we go down, 
# each following analysis has only one added restriction it cannot have the top 
# card from the previous straight, since then it would be the previous straight.
# Nevertheless, the loop is simply decorative, it could also be written as: 
# 1 * best straight cases + 9 * other straight cases
# Disclaimer: When we say random card, it means a random card that satisfies 
# the restrictions (the global restriction and of each case it's split into).
for i in range(10):
    # Cases with no pairs in the straight
    # 4 suits for each straight card * 2 random cards
    st_cases += 4**5 * comb(52 - 5*4 - 4 * min(i, 1), 2)
    # Cases with one pair in the straight
    # 4 suits for each straight card minus the paired one * paired card * suits for the pair * random card
    st_cases += 4**4 * comb(5, 1) * comb(4, 2) * comb(52 - 5*4 - 4 * min(i, 1), 1)
    # Cases with two pairs in the straight
    # 4 suits for each straight card minus the paired ones * 2 paired cards * suits for the pairs
    st_cases += 4**3 * comb(5, 2) * comb(4, 2)**2
    # Cases with a three of a kind in the straight
    # 4 suits for each straight card minus the three of a kind * three of a kind * suits for the other 2 cards
    st_cases += 4**4 * comb(5, 1) * comb(4, 3)
    
# We need to remove cases with flushes
for i in range(10):
    # Cases with no pairs in the straight
    st_cases -= 4 * (
        # Preamble: All cards from the straight have the flushed suit
        # 2 random cards (the straight is already flushed)
        # Total flushed cards: 5 (+ 2 optional)
        comb(52 - 5*4 - 4 * min(i, 1), 2)
        # Preamble: 1 card from the straight does not have the flushed suit
        # flushed extra card * random card * unflushed card * 3 suits for the unflushed card
        # Total flushed cards: 4 + 1 (+ 1 optional)
        + (
            # We have to divide the cases into two, since the unflushed card it 
            # is important to discern between 2 random extra flushed cards and 
            # 1 random extra flushed card and 1 unflushed card. Otherwise, we 
            # would be counting some cases twice.
            # Total flushed cards: 4 + 1
            comb(13 - 5 - min(i, 1), 1) * comb(39 - 5*3 - 3 * min(i, 1), 1) * comb(5, 1) * 3
            # Total flushed cards: 4 + 2
            + comb(13 - 5 - min(i, 1), 2) * comb(5, 1) * 3
        )
        # Preamble: 2 cards from the straight do not have the flushed suit
        # 2 flushed extra cards * 2 unflushed cards * 3 suits for the unflushed cards
        # Total flushed cards: 3 + 2
        + comb(13 - 5 - min(i, 1), 2) * comb(5, 2) * 3**2
    )

    
    # Cases with one pair in the straight
    st_cases -= 4 * (
        # Preamble: All cards from the straight have the flushed suit
        # random card * paired card * 3 suits for that paired card (the entire straight is flushed)
        # Total flushed cards: 5 (+ 1 optional)
        comb(52 - 5*4 - 4 * min(i, 1), 1) * comb(5, 1) * 3
        # Preamble: 1 card from the straight does not have the flushed suit
        # flushed extra card * paired card * (3 suits for the paired card * 3 suits for the unflushed card or 3 suits choose 2 [unflushed is paired])
        # Total flushed cards: 4 + 1
        + comb(13 - 5 - min(i, 1), 1) * comb(5, 1) * (comb(4, 1) * 3**2 + comb(3, 2)) # 4 + 1
    )
    
    # Cases with two pairs in the straight
    st_cases -= 4 * (
        # Preamble: All cards from the straight have the flushed suit
        # 2 paired cards * 3 suits for each paired cards
        comb(5, 2) * 3 * 3
    )
    
    # Cases with a three of a kind in the straight
    st_cases -= 4 * (
        # Preamble: All cards from the straight have the flushed suit
        # 1 three of a kind * 3 chose 2 suits for the other 2 cards
        comb(5, 1) * comb(3, 2)
    )
    
st_cases # Correct

6180020

In [20]:
# Poker cases
pk_cases = 13 * comb(52 - 4, 3)
pk_cases

224848

In [29]:
comb(52 - 2*4, 2)

946

In [31]:
comb(52 - 2*4, 1) * comb(52 - 3*4, 1) // comb(2, 1)

880

In [38]:
# Full house cases
fh_cases = (
    # three of a kind * pair * 2 extra cards (not paired)
    comb(13, 1) * comb(12, 1) * comb(4, 3) * comb(4, 2) * comb(11, 2) * 4**2
    # three of a kind * 2 pair
    + comb(13, 1) * comb(12, 2) * comb(4, 3) * comb(4, 2)**2
    # two three of a kinds * extra card
    + comb(13, 2) * comb(4, 3)**2 * comb(52 - 2*4, 1)
)
fh_cases # Correct

3473184

In [45]:
# Three of a kind cases
# (5 unique numbers - straights) * three of a kind * suits for tok * suits for the rest (without forming flush)
tok_cases = (comb(13, 5) - 10) * comb(5, 1) * comb(4, 3) * (4**4 - 3)
tok_cases

6461620

In [47]:
accum = rf_cases + sf_cases + fl_cases + st_cases + pk_cases + fh_cases + tok_cases
comb(52, 7) - accum

113387864

In [56]:
# Two pairs cases
tp_cases = (
    # Preamble: Only two pair
    # (5 unique numbers - straights) * 2 pairs * suit combinations
    (comb(13, 5) - 10) * comb(5, 2) * (
        # Both pairs have the same suit
        # 2 suits * (3 extra suits - flush cases)
        comb(4, 2) * (4**3 - 2)
        # 1 common suit between the pairs
        # 2 suits * 1 suit * (3 extra suits - flush cases)
        + comb(4, 1) * comb(4, 2) * (4**3 - 1)
        # No common suits between the pairs (choosing for the first pair is enough to determine all suits)
        # 2 suits * 3 extra suits 
        + comb(4, 2) * 4**3
    )
    # Preamble: 3 pair (no possible flushes or straights)
    # 4 unique numbers * 3 of them are pairs * suits for the pairs * extra suit
    + comb(13, 4) * comb(4, 3) * comb(4, 2)**3 * 4
)
tp_cases

31433400

In [65]:
(4**5 - 4 - 2 * comb(5, 4) * 3)

990

In [68]:
# One pair cases
# (6 unique numbers - straights) * pair * suits for the pair * suits for the rest (without forming flush)
pr_cases = (comb(13, 6) - (9 * 7 + 8)) * comb(6, 1) * comb(4, 2) * (
    # All combinations - 5 flushes without pair - flushes with pair
    4**5 - 4 - 2 * comb(5, 4) * 3
)
pr_cases

58627800

In [77]:
fl_cases

4089228

In [81]:
# High card cases
hc_cases = comb(52, 7) - (rf_cases + sf_cases + fl_cases + st_cases + pk_cases + fh_cases + tok_cases + tp_cases + pr_cases)
hc_cases # Correct

23294460

In [95]:
# Alternatively
# (7 unique numbers - possible straights) * (all suits - flushes)
hc_cases = (comb(13, 7) - (comb(8, 2) + 9*comb(7, 2))) * (4**7 - 4*(comb(7, 5)*3**2 + comb(7,6)*3 + comb(7,7)))
hc_cases # Correct
# Having a way to calculate each case separately allows us to avoid calculating one of them when we start applying them to the specific cases
# There is also the possibility of calculating them negatively, i.e. 
# calculating all cases and then removing several hands we already calculated 
# and removing remaining invalid hands

23294460